In [6]:
from netaddr import IPNetwork, IPAddress
import matplotlib.pyplot as plt

#c
#CITY_BLOCKS_PATH    = ''
#CITY_LOCATIONS_PATH = ''
#DATA_PATHS          = ''

#m
CITY_BLOCKS_PATH    = '/FileStore/tables/5ne9pwzz1484570889985/GeoLite2_City_Blocks_IPv4-82d63.csv'
CITY_LOCATIONS_PATH = '/FileStore/tables/fcj10i6i1484571614099/GeoLite2_City_Locations_en-6f8fe.csv'
DATA_PATHS          = ['/FileStore/tables/xqubiq301484572436491/NetworkTraffic100.csv',\
                       '/FileStore/tables/810vguox1485019521316/100000am.csv',\
                       '/FileStore/tables/810vguox1485019521316/100000pm.csv']
#DATA_PATHS          = ['/FileStore/tables/xqubiq301484572436491/NetworkTraffic100.csv']

CITY_BLOCKS_PATH    = 'geolite/GeoLite2-City-Blocks-IPv4.csv'
CITY_LOCATIONS_PATH = 'geolite/GeoLite2-City-Locations-en.csv'
DATA_PATHS          = ['data/NetworkTraffic100.csv',\
                       'data/100000am.csv',\
                       'data/100000pm.csv',\
                       'data/1000000am.csv',\
                       'data/1000000pm.csv']



#s
#CITY_BLOCKS_PATH    = ''/FileStore/tables/jk3parwb1484571151117/GeoLite2_City_Blocks_IPv4-82d63.csv''
#CITY_LOCATIONS_PATH = '/FileStore/tables/1rn58fwl1484571610700/GeoLite2_City_Locations_en-6f8fe.csv'
#DATA_PATHS          = '/FileStore/tables/tjpvf32z1484575566663/100.csv']

def isIpInNet(ip, net):
  """
  Checks if a ip is part of a net.
  e.g. isIpInNet("192.168.0.1", "192.168.0.0/24")
  """
  if IPAddress(ip) in IPNetwork(net):
    return True
  return False

def buildNetworkCountryMap():
  """
  Builds a map with data from geoIp2 (http://geolite.maxmind.com/download/geoip/database/GeoLite2-City.mmdb.gz)
  1st file contains networks and locationkey (much more data available but we consider only this)
  2nd file contains locationkey and country name.
  method joins this two datarecords
  
  returns an RDD as [('network', 'country'), ...]
  """
  geoipfile = sc.textFile(CITY_BLOCKS_PATH, 2)
  
  #(geoname_id, network) / filters the header
  geoipdata = geoipfile.map(lambda l:l.split(',')).filter(lambda l:l[0] not in 'network').map(lambda p: (p[1], p[0]))

  locationsfile = sc.textFile(CITY_LOCATIONS_PATH, 2)
  
  #(geoname_id, country_name) / filters the header
  locationsdata = locationsfile.map(lambda l:l.split(',')).filter(lambda l:l[0] not in 'geoname_id').map(lambda p: (p[0], p[5]))

  #join on geoname_id
  joineddata = geoipdata.join(locationsdata)
  
  #(network, country_name)
  networkmap = joineddata.map(lambda d: (d[1][0],d[1][1]))
  return networkmap

def getCountryByIpBc(ip):
  """
  Returns the country for the given ip or None if not found
  """
  firstoctet = ip.split(".")[0]  
  #For speed improvements we build a map with the first 3 digits of the ip. # "built" or move to actual creation of map
  
  if firstoctet in reducednetworkmapbc.value: 
    for network in reducednetworkmapbc.value[firstoctet]:
      if isIpInNet(ip, network[0]):
        return network[1]
  return None

def loadData(filePath):
  #return sc.textFile(filePath).map(lambda l: l.split(",")).filter(lambda l:l[0] not in 'ts' and len[l] > 10)
  return sc.textFile(filePath).map(lambda l: l.split(",")).filter(lambda l:l[0] not in 'ts' and len(l) > 10)
  
networkmap = buildNetworkCountryMap().persist()




In [7]:
#We build a map with the first part of the ip, eg. 255 and a list of all the networks (max 4.294.967.296 adresse w/ ipv4)
reducednetworkmap = networkmap.map(lambda e: (e[0].split(".")[0], [e])).reduceByKey(lambda a,b: a+b)

#We need to broadcast this to all nodes to be able to access this map from inside a map function
reducednetworkmapbc = sc.broadcast(reducednetworkmap.collectAsMap())

In [8]:
#read the firewall data
data = loadData(DATA_PATHS[0])
#print data.take(5)
for path in range(1, len(DATA_PATHS)):
  #data = sc.union(data, loadData(DATA_PATHS[path]))
  data = data.union(loadData(DATA_PATHS[path]))

# maybe create an RDD that only contains the relevant columns
sourceAddressIndex = 3
protocolIndex = 7
inBytesIndex = 12
outBytesIndex = 14
#data.take(5)

#get list of ips and country

#-> here sollten wir noch distinct machen auf der ip w������rde nochmals viel schneller werden, oder?!
# yep, vorallem da unsere 4.5s nur fuer take(5) gelten, bei take(100) sind's bereits 47s - von daher gute Optimierung ;o)
#ipCountry = data.map(lambda l: l.split(',')).map(lambda line: (line[3], getCountryByIpBc(line[3]))) #47s
#ipCountry = data.map(lambda l: l.split(',')).map(lambda line: line[3]).distinct().map(lambda sa: (sa, getCountryByIpBc(sa))) # 16s
#ipCountry.take(100)

#print networkmap.count() #2'766'452
#print reducednetworkmap.count() #221 => 2'766'452 / 221 = 12'500; 12'500 / 2 = 6'250 mean average (loops) after dictionary access instead of 2'766'452 / 2 = 1'383'226 (calculation supposes a uniform distribution - which is likely not the case)

In [9]:
# requests by country
requestsByCountry = data\
  .map(lambda line: line[sourceAddressIndex]).distinct()\
  .map(lambda sa: (getCountryByIpBc(sa), 1))\
  .filter(lambda r: r[0] is not None)\
  .reduceByKey(lambda a, b: a + b)\
  .takeOrdered(10, lambda x: -x[1]) # use takeOrdered for small results sets only

In [5]:
# visualize requests by country
fig = plt.figure(figsize=(10, 4))
plt.pie([r[1] for r in requestsByCountry],\
        explode=[0.1 if i == 0 else 0.0 for i, r in enumerate(requestsByCountry)],\
        labels=[r[0] for r in requestsByCountry],\
        autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')
fig.show()
#display(fig) 

In [18]:
# requests by port
# check for destination?
print data.filter(lambda r: r[sourceAddressIndex].startswith('194.'))\
  .map(lambda r: (r[protocolIndex], 1))\
  .reduceByKey(lambda a, b: a + b)\
  .takeOrdered(10, lambda x: -x[1])

[(u'ICMP', 12960), (u'UDP', 1053), (u'TCP', 986), (u'SCPS', 76)]


In [ ]:
# heavy senders in MiB
heavySenders = data.map(lambda r: (r[sourceAddressIndex], int(r[outBytesIndex])))\
  .reduceByKey(lambda a, b: a + b)\
  .takeOrdered(20, lambda x: -x[1])

print [(x[0], x[1] / 1024.0 / 1024.0) for x in heavySenders]

In [ ]:
# heavy receivers in MiB
heavyReceivers = data.map(lambda r: (r[sourceAddressIndex], int(r[inBytesIndex])))\
  .reduceByKey(lambda a, b: a + b)\
  .takeOrdered(20, lambda x: -x[1])

print [(x[0], x[1] / 1024.0 / 1024.0) for x in heavyReceivers]

In [9]:
# overall data whores in MiB
overallDataWhores = data.map(lambda r: (r[sourceAddressIndex], int(r[inBytesIndex]) + int(r[outBytesIndex])))\
  .reduceByKey(lambda a, b: a + b)\
  .takeOrdered(20, lambda x: -x[1])

print [(x[0], x[1] / 1024.0 / 1024.0) for x in overallDataWhores]

In [10]:
# senders and receivers
data\
  .filter(lambda r: int(r[inBytesIndex]) > 0 and int(r[outBytesIndex]) > 0)\
  .map(lambda r: (r[sourceAddressIndex], int(r[inBytesIndex]) + int(r[outBytesIndex])))\
  .take(10)